### Clase Neurona

In [4]:
class neurona(object):
    def __init__(self, f=0, c=0, dim=0):
        self.c = c
        self.f = f
        self.dim = dim
        self.w = normalize(np.random.rand(self.dim).reshape(1,-1))
        self.output = -10.0
        self.label = -1
            
    def predict(self, input):
        self.output = input @ self.w.T
        return self.output
    
    def fit(self, input, alfa=1):
        self.w = normalize(self.w + (alfa*input))
            
    def labeling(self, inputs, targets):
        Y = inputs @ self.w.T
        self.label = targets[np.argmax(Y)]

### Clase SOM

In [79]:
from tqdm import tqdm
class som():
    
    def __init__(self, filas=1, columnas=1, dim=1, init_radious=0, init_alfa=1):
        self.lista_neuronas = []
        self.filas = filas
        self.columnas = columnas
        self.dim = dim
        self.labels = []
        self.radious = init_radious
        self.init_alfa = init_alfa
        self.t = 0
        # Considera que un mapa rectangular es una lista de objetos "neurona", que viene localizado por sus atributos "fila" y "columna"
        for fila in range(self.filas):
            for columna in range(self.columnas):
                self.lista_neuronas.append(neurona(f=fila, c=columna, dim=dim))
                
    def fit(self, inputs, max_epochs=1):
        # método similar a otros algoritmo de ML. Recibe las entradas, el radio inicial, el factor de apendizaje inicial,
        # el máximo de épocas y devuelve los pesos ajustados
        P = inputs.shape[0]
        for epoch in range(max_epochs):
            for x in tqdm(inputs, desc="Epoch:"+str(epoch)+"   R:"+str(self.radious)+"     " ,bar_format='[{elapsed}<{remaining}] {n_fmt}/{total_fmt} | {l_bar}{bar} {rate_fmt}{postfix}'):
                alfa = self.init_alfa/(1.0 + float(self.t/P))
                i_gana, y_gana = -1, float('-inf')
                for i in range(self.filas*self.columnas):
                    y_predict = self.lista_neuronas[i].predict(x.reshape(1,-1))
                    if y_predict > y_gana:
                        y_gana = y_predict
                        i_gana = i
                f_gana = int(i_gana / self.columnas)
                c_gana = i_gana % self.columnas
                
                # Conjunto de vecinas para un radious
                for f in range(f_gana - self.radious, f_gana + self.radious+1):
                    if f < 0:
                        row = self.filas + f
                    else:
                        if f > self.filas-1:
                            row = f % self.filas
                        else:
                            row = f

                    for c in range(c_gana - self.radious, c_gana + self.radious+1):
                        if c < 0:
                            column = self.columnas + c 
                        else:
                            if c > self.columnas-1:
                                column = c % self.columnas
                            else:
                                column = c
                        self.lista_neuronas[(row*self.columnas) + column].fit(x.reshape(1,-1), alfa)
                self.t += 1
                # if (t%1000) == 0:
                #    print(t, self.radious, "  ", end='')
            if self.radious > 0:
                self.radious -= 1
                            
    def neuron_labeling(self, inputs, target):
        # recorre la lista de neuronas y va llamanado a su metodo de etiquetado para cada neurona        
        for i in range(self.filas*self.columnas):
            self.lista_neuronas[i].labeling(inputs, target)
 
    def predict(self, inputs):
        # recorre la lista de neuronas y calcula la salida de un conjunto de muestras
        # util para usar la salida del som como entrada a otrso sistemas
        output_list = []
        for x in tqdm(inputs):
            for i in range(self.filas*self.columnas):
                output_list.append(self.lista_neuronas[i].predict(x.reshape(1,-1)))
        return np.array(output_list).reshape(inputs.shape[0], -1)    
    
    def label_predict(self, inputs):
        return np.array([self.lista_neuronas[i].label for i in np.argmax(self.predict(inputs),axis=1)])
        
    def accuracy(self, inputs, targets):
        return np.mean(self.label_predict(inputs) == targets)
          